# Library Imports

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import joblib
import json

# Load Dataset

In [33]:
from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/AI_ML_datasets/energy_data_set.csv'
df = pd.read_csv(file_path)

print(f"Dataset loaded successfully!")
print(f"Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset loaded successfully!
Shape: (19735, 29)
Columns: ['date', 'Appliances', 'lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4', 'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'T9', 'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility', 'Tdewpoint', 'rv1', 'rv2']


# Data Type Conversion

In [34]:
# Convert date column to datetime
df['date'] = pd.to_datetime(df['date'])
print(" Date column converted to datetime")

 Date column converted to datetime


# Time-based Feature Engineering

In [35]:
# Extract time-based features
df['hour'] = df['date'].dt.hour
df['minute'] = df['date'].dt.minute
df['day'] = df['date'].dt.day
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

# Create NSM (Number of Seconds since Midnight)
df['NSM'] = df['date'].dt.hour * 3600 + df['date'].dt.minute * 60 + df['date'].dt.second

# Adjust the weekday so that 0 = Sunday, 6 = Saturday
df['day_of_week'] = (df['date'].dt.dayofweek + 1) % 7

# Create WeekStatus
df['WeekStatus'] = df['day_of_week'].apply(lambda x: 'Weekend' if x >= 5 else 'Weekday')

print("Time-based features created successfully!")
print(f"New columns: {['hour', 'minute', 'day', 'month', 'year', 'NSM', 'day_of_week', 'WeekStatus']}")

Time-based features created successfully!
New columns: ['hour', 'minute', 'day', 'month', 'year', 'NSM', 'day_of_week', 'WeekStatus']


# Missing Values Analysis and Handling

In [36]:
print("=== MISSING VALUES ANALYSIS ===")
missing_values = df.isnull().sum()
missing_percent = (missing_values / len(df)) * 100

missing_df = pd.DataFrame({
    'Column': missing_values.index,
    'Missing_Count': missing_values.values,
    'Missing_Percentage': missing_percent.values
})

# Show only columns with missing values
missing_df_filtered = missing_df[missing_df['Missing_Count'] > 0]

if len(missing_df_filtered) > 0:
    print("Columns with missing values:")
    print(missing_df_filtered)


else:
    print("No missing values found in the dataset!")




=== MISSING VALUES ANALYSIS ===
No missing values found in the dataset!


# Outlier Detection

In [37]:
print("=== OUTLIER DETECTION AND ANALYSIS ===")

def detect_outliers_iqr(df, column):
    """Detect outliers using IQR method"""
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

def detect_outliers_zscore(df, column, threshold=3):
    """Detect outliers using Z-score method"""
    z_scores = np.abs(stats.zscore(df[column]))
    outliers = df[z_scores > threshold]
    return outliers

# Analyze outliers in target variable (Appliances)
appliances_outliers_iqr, lower_bound, upper_bound = detect_outliers_iqr(df, 'Appliances')
appliances_outliers_zscore = detect_outliers_zscore(df, 'Appliances')

print(f"Appliances outliers (IQR method): {len(appliances_outliers_iqr)} ({len(appliances_outliers_iqr)/len(df)*100:.2f}%)")
print(f"Appliances outliers (Z-score method): {len(appliances_outliers_zscore)} ({len(appliances_outliers_zscore)/len(df)*100:.2f}%)")
print(f"IQR bounds: Lower={lower_bound:.2f}, Upper={upper_bound:.2f}")

=== OUTLIER DETECTION AND ANALYSIS ===
Appliances outliers (IQR method): 2138 (10.83%)
Appliances outliers (Z-score method): 540 (2.74%)
IQR bounds: Lower=-25.00, Upper=175.00


# Outlier Treatment Strategy

In [38]:
print("=== OUTLIER TREATMENT STRATEGY ===")

# Analyze impact of outliers
print("Appliances statistics with outliers:")
print(df['Appliances'].describe())

# Remove outliers for comparison
df_no_outliers = df[~df.index.isin(appliances_outliers_iqr.index)]
print(f"\nAppliances statistics without outliers:")
print(df_no_outliers['Appliances'].describe())

print(f"\nImpact of outlier removal:")
print(f"Original dataset size: {len(df)}")
print(f"After outlier removal: {len(df_no_outliers)}")
print(f"Data lost: {len(df) - len(df_no_outliers)} records ({(len(df) - len(df_no_outliers))/len(df)*100:.2f}%)")

=== OUTLIER TREATMENT STRATEGY ===
Appliances statistics with outliers:
count    19735.000000
mean        97.694958
std        102.524891
min         10.000000
25%         50.000000
50%         60.000000
75%        100.000000
max       1080.000000
Name: Appliances, dtype: float64

Appliances statistics without outliers:
count    17597.000000
mean        67.209752
std         28.480499
min         10.000000
25%         50.000000
50%         60.000000
75%         80.000000
max        170.000000
Name: Appliances, dtype: float64

Impact of outlier removal:
Original dataset size: 19735
After outlier removal: 17597
Data lost: 2138 records (10.83%)


# Keep outliers but cap extreme values

In [39]:
def cap_outliers(series, lower_percentile=1, upper_percentile=99):
    """Cap outliers at specified percentiles"""
    lower_cap = series.quantile(lower_percentile/100)
    upper_cap = series.quantile(upper_percentile/100)
    return series.clip(lower=lower_cap, upper=upper_cap)

# Apply capping to Appliances
df['Appliances_capped'] = cap_outliers(df['Appliances'])

print(f"\n Outlier treatment applied: Capping at 1st and 99th percentiles")
print(f"Capped statistics:")
print(df['Appliances_capped'].describe())

# Use capped version as target
df['Appliances'] = df['Appliances_capped']
df.drop('Appliances_capped', axis=1, inplace=True)


 Outlier treatment applied: Capping at 1st and 99th percentiles
Capped statistics:
count    19735.000000
mean        96.698090
std         96.824925
min         20.000000
25%         50.000000
50%         60.000000
75%        100.000000
max        576.600000
Name: Appliances_capped, dtype: float64


We decided to cap outliers at the 1st and 99th percentiles to balance between outlier impact mitigation and data preservation. This ensures:
No data loss (unlike removal),
No artificial bias (unlike imputation),
More stable and generalizable model training.

# Data Scaling and Normalization

In [40]:
print("=== DATA SCALING AND NORMALIZATION ===")

# Identify numeric columns for scaling
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# Remove non-feature columns
exclude_cols = ['Appliances', 'NSM', 'day_of_week', 'hour', 'minute', 'day', 'month', 'year']
feature_cols = [col for col in numeric_cols if col not in exclude_cols]

print(f"Columns to be scaled: {feature_cols}")

# Apply chosen scaling method to all features
print("=== APPLYING SCALING TO DATASET ===")

# Create a copy for scaling
df_scaled = df.copy()

# Initialize scaler
scaler = StandardScaler()

# Scale feature columns
df_scaled[feature_cols] = scaler.fit_transform(df[feature_cols])

print(" Scaling applied successfully!")
print(f"Scaled features: {len(feature_cols)} columns")

# Verify scaling worked
print("\nBefore scaling (first 5 features):")
print(df[feature_cols[:5]].describe())

print("\nAfter scaling (first 5 features):")
print(df_scaled[feature_cols[:5]].describe())

# Save scaler for later use
joblib.dump(scaler, 'feature_scaler.pkl')
print(" Scaler saved as 'feature_scaler.pkl'")

=== DATA SCALING AND NORMALIZATION ===
Columns to be scaled: ['lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4', 'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'T9', 'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility', 'Tdewpoint', 'rv1', 'rv2']
=== APPLYING SCALING TO DATASET ===
 Scaling applied successfully!
Scaled features: 27 columns

Before scaling (first 5 features):
             lights            T1          RH_1            T2          RH_2
count  19735.000000  19735.000000  19735.000000  19735.000000  19735.000000
mean       3.801875     21.686571     40.259739     20.341219     40.420420
std        7.935988      1.606066      3.979299      2.192974      4.069813
min        0.000000     16.790000     27.023333     16.100000     20.463333
25%        0.000000     20.760000     37.333333     18.790000     37.900000
50%        0.000000     21.600000     39.656667     20.000000     40.500000
75%        0.000000     22.600000     43.066667    

# Temporal Data Splitting

In [41]:
print("=== TEMPORAL DATA SPLITTING ===")

# Sort by date to ensure temporal order
df_scaled = df_scaled.sort_values('date').reset_index(drop=True)

print(f"Dataset date range:")
print(f"Start: {df_scaled['date'].min()}")
print(f"End: {df_scaled['date'].max()}")
print(f"Total records: {len(df_scaled)}")

# Calculate split point (80% for training, 20% for testing)
split_point = int(len(df_scaled) * 0.8)

# Split data temporally (NO RANDOM SHUFFLING!)
train_data = df_scaled[:split_point].copy()
test_data = df_scaled[split_point:].copy()

print(f"\nTraining set:")
print(f"  Size: {len(train_data)} records")
print(f"  Date range: {train_data['date'].min()} to {train_data['date'].max()}")

print(f"\nTest set:")
print(f"  Size: {len(test_data)} records")
print(f"  Date range: {test_data['date'].min()} to {test_data['date'].max()}")

# Prepare feature matrices and target vectors
X_train = train_data[feature_cols]
y_train = train_data['Appliances']
X_test = test_data[feature_cols]
y_test = test_data['Appliances']

print(f"\nFeature matrix shapes:")
print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_train: {y_train.shape}")
print(f"y_test: {y_test.shape}")

=== TEMPORAL DATA SPLITTING ===
Dataset date range:
Start: 2016-01-11 17:00:00
End: 2016-05-27 18:00:00
Total records: 19735

Training set:
  Size: 15788 records
  Date range: 2016-01-11 17:00:00 to 2016-04-30 08:10:00

Test set:
  Size: 3947 records
  Date range: 2016-04-30 08:20:00 to 2016-05-27 18:00:00

Feature matrix shapes:
X_train: (15788, 27)
X_test: (3947, 27)
y_train: (15788,)
y_test: (3947,)


# Data Split Validation

In [42]:
print("=== DATA SPLIT VALIDATION ===")

# Check for data leakage (critical!)
print("Checking for data leakage...")
train_max_date = train_data['date'].max()
test_min_date = test_data['date'].min()

if train_max_date < test_min_date:
    print(" NO DATA LEAKAGE: Training data ends before test data begins")
    print(f"   Training ends: {train_max_date}")
    print(f"   Testing begins: {test_min_date}")
else:
    print(" DATA LEAKAGE DETECTED!")
    print("   This will cause overly optimistic results!")

# Statistical comparison between train and test sets
print(f"\n=== STATISTICAL COMPARISON ===")
print("Target variable (Appliances) statistics:")
print(f"Training set - Mean: {y_train.mean():.2f}, Std: {y_train.std():.2f}")
print(f"Test set - Mean: {y_test.mean():.2f}, Std: {y_test.std():.2f}")

# Feature distribution comparison
print(f"\nFeature distribution comparison (sample features):")
sample_features = ['T1', 'T_out', 'RH_1', 'lights']
for feature in sample_features:
    train_mean = train_data[feature].mean()
    test_mean = test_data[feature].mean()
    print(f"{feature} - Train: {train_mean:.2f}, Test: {test_mean:.2f}, Diff: {abs(train_mean-test_mean):.2f}")

=== DATA SPLIT VALIDATION ===
Checking for data leakage...
 NO DATA LEAKAGE: Training data ends before test data begins
   Training ends: 2016-04-30 08:10:00
   Testing begins: 2016-04-30 08:20:00

=== STATISTICAL COMPARISON ===
Target variable (Appliances) statistics:
Training set - Mean: 96.93, Std: 99.05
Test set - Mean: 95.79, Std: 87.36

Feature distribution comparison (sample features):
T1 - Train: -0.33, Test: 1.33, Diff: 1.66
T_out - Train: -0.29, Test: 1.17, Diff: 1.46
RH_1 - Train: -0.06, Test: 0.22, Diff: 0.28
lights - Train: 0.06, Test: -0.26, Diff: 0.32


# Feature Correlation Analysis (for preprocessing validation)

In [43]:
print("=== FEATURE CORRELATION ANALYSIS ===")

# Calculate correlation matrix for training data only
train_corr_matrix = train_data[feature_cols + ['Appliances']].corr()

# Top correlations with target variable
target_correlations = train_corr_matrix['Appliances'].abs().sort_values(ascending=False)
print("Top 10 features correlated with Appliances:")
print(target_correlations[1:11])  # Exclude self-correlation

# Highly correlated feature pairs (potential multicollinearity)
def find_high_correlations(corr_matrix, threshold=0.8):
    high_corr_pairs = []
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            corr_val = abs(corr_matrix.iloc[i, j])
            if corr_val > threshold:
                high_corr_pairs.append((corr_matrix.columns[i],
                                      corr_matrix.columns[j],
                                      corr_val))
    return high_corr_pairs

high_corr = find_high_correlations(train_corr_matrix[feature_cols].corr(), 0.8)
print(f"\nHighly correlated feature pairs (>0.8):")
for pair in high_corr[:10]:  # Show top 10
    print(f"{pair[0]} - {pair[1]}: {pair[2]:.3f}")

if len(high_corr) > 0:
    print(f"\n  Found {len(high_corr)} highly correlated pairs - consider feature selection!")
else:
    print("\n No highly correlated features found")

=== FEATURE CORRELATION ANALYSIS ===
Top 10 features correlated with Appliances:
lights    0.223721
T2        0.162093
RH_out    0.157957
T6        0.128450
T3        0.126982
RH_8      0.116507
T_out     0.112589
RH_1      0.096134
T1        0.089316
RH_6      0.083196
Name: Appliances, dtype: float64

Highly correlated feature pairs (>0.8):
T1 - T2: 0.865
T1 - T3: 0.974
T1 - T4: 0.966
T1 - T5: 0.974
T1 - RH_6: 0.890
T1 - T7: 0.936
T1 - T8: 0.936
T1 - T9: 0.946
RH_1 - RH_2: 0.928
RH_1 - RH_3: 0.954

  Found 66 highly correlated pairs - consider feature selection!


# Preprocessing Summary Report

In [44]:
print("=" * 60)
print("           DATA PREPROCESSING SUMMARY REPORT")
print("=" * 60)

print(f"\n DATASET OVERVIEW:")
print(f"   • Original dataset size: {df.shape[0]:,} records, {df.shape[1]} features")
print(f"   • Date range: {df['date'].min()} to {df['date'].max()}")
print(f"   • Sampling interval: 10 minutes")
print(f"   • Duration: {(df['date'].max() - df['date'].min()).days} days")

print(f"\n DATA QUALITY ASSESSMENT:")
missing_count = df.isnull().sum().sum()
print(f"   • Missing values: {missing_count} total")
print(f"   • Outliers detected: {len(appliances_outliers_iqr)} ({len(appliances_outliers_iqr)/len(df)*100:.1f}%)")
print(f"   • Outlier treatment: Capping applied")

print(f"\n FEATURE ENGINEERING:")
print(f"   • Time-based features created: hour, day_of_week, WeekStatus, NSM")
print(f"   • Temperature features: {len([c for c in feature_cols if c.startswith('T')])}")
print(f"   • Humidity features: {len([c for c in feature_cols if c.startswith('RH')])}")
print(f"   • Weather features: 3 (Windspeed, Visibility, Press_mm_hg)")
print(f"   • Total features for modeling: {len(feature_cols)}")

print(f"\n DATA SCALING:")
print(f"   • Method: StandardScaler (z-score normalization)")
print(f"   • Reason: Different feature scales, suitable for neural networks")
print(f"   • Features scaled: {len(feature_cols)}")

print(f"\n TEMPORAL DATA SPLIT:")
print(f"   • Training set: {len(X_train):,} records ({len(X_train)/len(df)*100:.1f}%)")
print(f"   • Test set: {len(X_test):,} records ({len(X_test)/len(df)*100:.1f}%)")
print(f"   • Split method: Temporal (no shuffling)")
print(f"   • Data leakage: {' None detected' if train_max_date < test_min_date else ' DETECTED!'}")

print(f"\n TARGET VARIABLE (Appliances):")
print(f"   • Training mean: {y_train.mean():.2f} Wh")
print(f"   • Training std: {y_train.std():.2f} Wh")
print(f"   • Range: {y_train.min():.0f} - {y_train.max():.0f} Wh")

print(f"\n KEY CORRELATIONS WITH TARGET:")
top_5_corr = target_correlations[1:6]
for feature, corr in top_5_corr.items():
    print(f"   • {feature}: {corr:.3f}")

print(f"\n PREPROCESSING STATUS: COMPLETE")

           DATA PREPROCESSING SUMMARY REPORT

 DATASET OVERVIEW:
   • Original dataset size: 19,735 records, 37 features
   • Date range: 2016-01-11 17:00:00 to 2016-05-27 18:00:00
   • Sampling interval: 10 minutes
   • Duration: 137 days

 DATA QUALITY ASSESSMENT:
   • Missing values: 0 total
   • Outliers detected: 2138 (10.8%)
   • Outlier treatment: Capping applied

 FEATURE ENGINEERING:
   • Time-based features created: hour, day_of_week, WeekStatus, NSM
   • Temperature features: 11
   • Humidity features: 10
   • Weather features: 3 (Windspeed, Visibility, Press_mm_hg)
   • Total features for modeling: 27

 DATA SCALING:
   • Method: StandardScaler (z-score normalization)
   • Reason: Different feature scales, suitable for neural networks
   • Features scaled: 27

 TEMPORAL DATA SPLIT:
   • Training set: 15,788 records (80.0%)
   • Test set: 3,947 records (20.0%)
   • Split method: Temporal (no shuffling)
   • Data leakage:  None detected

 TARGET VARIABLE (Appliances):
   • Tr

# Save Preprocessed Data

In [45]:
print("=== SAVING PREPROCESSED DATA ===")

# Save datasets
train_data.to_csv('train_data_preprocessed.csv', index=False)
test_data.to_csv('test_data_preprocessed.csv', index=False)


# Save feature names
with open('feature_names.txt', 'w') as f:
    for feature in feature_cols:
        f.write(f"{feature}\n")

# Save preprocessing metadata
preprocessing_info = {
    'feature_columns': feature_cols,
    'scaler_type': 'StandardScaler',
    'split_point': split_point,
    'train_size': len(X_train),
    'test_size': len(X_test),
    'outlier_treatment': 'capping_1_99_percentiles',
    'date_range_train': [str(train_data['date'].min()), str(train_data['date'].max())],
    'date_range_test': [str(test_data['date'].min()), str(test_data['date'].max())]
}

with open('preprocessing_info.json', 'w') as f:
    json.dump(preprocessing_info, f, indent=2)

print(" Saved files:")
print("   • train_data_preprocessed.csv")
print("   • test_data_preprocessed.csv")





=== SAVING PREPROCESSED DATA ===
 Saved files:
   • train_data_preprocessed.csv
   • test_data_preprocessed.csv
